In [6]:
import os

import sys
import numpy as np
import pandas as pd
import importlib

import McsPy
import McsPy.McsData
import McsPy.McsCMOS
from McsPy import ureg, Q_

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

from scipy.signal import butter, lfilter, freqz, find_peaks, correlate, gaussian, filtfilt
from scipy import stats
from scipy import signal

from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing


import glob
import pickle
from pathlib import Path
import qgrid
import plotly.express as px
%matplotlib inline


In [7]:
input_directory = r"D:\Files_Reutlingen_Jenny\19-04-15\190415_paper"
output_directory = r"D:\Files_Reutlingen_Jenny\19-04-15\190415_paper"
temp_dir = r"D:\Files_Reutlingen_Jenny\19-04-15\190415_paper\spike_extraction"

In [8]:
os.chdir(input_directory)

In [9]:
filename = "190415_03_Cortex-synChR2-A_aCSF_rewashafterhcsf"

In [10]:
# import the MAIN Dictionary
MAIN_RECORDING_DICTIONARY = pickle.load(open(
    os.path.join(output_directory+'\\MAIN_RECORDING_Dictionary_'+filename+'.pkl'),
    "rb")
    )


network_bursts_seconds = MAIN_RECORDING_DICTIONARY['network_bursts_seconds']
spikedic_MAD = MAIN_RECORDING_DICTIONARY['spikedic_MAD']
fr_dic = MAIN_RECORDING_DICTIONARY['fr_dic']
isi_dictionary = MAIN_RECORDING_DICTIONARY['isi_dictionary']
isi_average_dic = MAIN_RECORDING_DICTIONARY['isi_average_dic']
Infos_Recording = MAIN_RECORDING_DICTIONARY['Infos_Recording']
Infos_Analysis = MAIN_RECORDING_DICTIONARY['Infos_Analysis']
Infos_Anatomy = MAIN_RECORDING_DICTIONARY['Infos_Anatomy']
Bursts = MAIN_RECORDING_DICTIONARY['Bursts']
Interburst_Intervals = MAIN_RECORDING_DICTIONARY['Interburst-Intervals']
bursting_time_per_channel = MAIN_RECORDING_DICTIONARY['bursting_time_per_channel']
bursts_per_channel = MAIN_RECORDING_DICTIONARY['bursts_per_channel']
burst_connections = MAIN_RECORDING_DICTIONARY['burst_connections']
Basics = MAIN_RECORDING_DICTIONARY['Basics']
inverted_layerdic = Infos_Anatomy['layerdic_invert']
layerdic = Infos_Anatomy['layerdic']
tick = 40
scale_factor_for_second = 1e-06
timelengthrecording_s = Infos_Recording['timelengthrecording_s']


In [11]:
folderlist = glob.glob('*DATAFRAME*.pkl')

In [12]:
df_from_each_file = (pickle.load(open(Path(input_directory, filename), "rb")) for filename in folderlist)
df   = pd.concat(df_from_each_file, ignore_index=True)

In [13]:
df.iloc[3]['network_burst_seconds']

'(27.22547789022808, 27.285476937341837)'

# Rasterplots
First, we want to be sure to create a rasterplot of the current networkburst.

Important: We need to be sure the Rasterplot always shows every channel that occurs throughout the whole recording analysis.

In [16]:
df

,filename,recording_date,timelength_recording_s,firingrate_whole_recording_Hz,active_channels_whole_recording,network_burst_seconds,timelength_network_burst_s,bursting_channels,number_of_bursting_channels,number_burst_starts,...,isi_std_layer1,isi_std_layer23,isi_std_layer4,isi_std_layer56,isi_std_whitematter',channels_covered_layer1,channels_covered_layer23,channels_covered_layer4,channels_covered_layer56,channels_covered_whitematter
0,190415_01_Cortex-synChR2-A_aCSF_nodrug,2019-04-15,1800.0,385.713,170,"(0.005639999999999999, 11.990568622864936)",11.984929,"[A11, A14, B11, B14, B5, C11, C12, C13, C15, C...",90,286,...,6.743655,20.920045,53.910257,25.461226,29.087022,5,71,28,125,26
1,190415_01_Cortex-synChR2-A_aCSF_nodrug,2019-04-15,1800.0,385.713,170,"(12.725564245543762, 19.50052330121305)",6.774959,"[A14, B10, B11, C11, C12, C6, C7, D11, D12, D6...",67,159,...,7.431367,13.925208,45.711174,18.092785,19.801508,5,71,28,125,26
2,190415_01_Cortex-synChR2-A_aCSF_nodrug,2019-04-15,1800.0,385.713,170,"(19.60052330121305, 26.710480957330674)",7.109958,"[A10, A11, B10, B11, B12, B5, C11, C6, C7, D15...",73,154,...,5.790017,12.833161,46.022109,19.547113,16.487408,5,71,28,125,26
3,190415_01_Cortex-synChR2-A_aCSF_nodrug,2019-04-15,1800.0,385.713,170,"(27.22547789022808, 27.285476937341837)",0.059999,"[D7, H4, M2]",3,3,...,0.000000,0.386896,NaN,0.664970,NaN,5,71,28,125,26
4,190415_01_Cortex-synChR2-A_aCSF_nodrug,2019-04-15,1800.0,385.713,170,"(27.38547693734184, 29.975461512495798)",2.589985,"[B11, C11, C13, C6, D11, D6, D7, D8, E10, E11,...",44,54,...,5.115923,10.118248,20.101840,10.753830,7.793206,5,71,28,125,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2644,190415_03_Cortex-synChR2-A_aCSF_rewashafterhcsf,2019-04-15,1800.0,368.373,175,"(1790.066811618301, 1790.316808387755)",0.249997,"[C11, D7, G6, K3, M3, M4, N2, N3]",8,5,...,0.039200,0.881307,0.000000,0.978210,1.492000,5,71,28,125,26
2645,190415_03_Cortex-synChR2-A_aCSF_rewashafterhcsf,2019-04-15,1800.0,368.373,175,"(1790.4168083877548, 1791.7307953363465)",1.313987,"[A11, B7, D6, D7, D8, E10, E11, E4, F8, G3, G6...",24,18,...,4.274750,4.154880,6.987447,6.621121,5.583673,5,71,28,125,26
2646,190415_03_Cortex-synChR2-A_aCSF_rewashafterhcsf,2019-04-15,1800.0,368.373,175,"(1791.8307953363465, 1792.876119021114)",1.045324,"[C12, C7, D11, D5, D6, D8, E9, F10, F13, G8, G...",25,26,...,6.185967,5.006054,9.712707,6.020411,4.195919,5,71,28,125,26
2647,190415_03_Cortex-synChR2-A_aCSF_rewashafterhcsf,2019-04-15,1800.0,368.373,175,"(1792.9694514929683, 1792.8927846109318)",-0.076667,[],0,0,...,NaN,NaN,NaN,NaN,NaN,5,71,28,125,26


In [17]:
df.iloc[2648]

filename                             190415_03_Cortex-synChR2-A_aCSF_rewashafterhcsf
recording_date                                                            2019-04-15
timelength_recording_s                                                        1800.0
firingrate_whole_recording_Hz                                                368.373
active_channels_whole_recording                                                  175
network_burst_seconds                       (1792.9927846109317, 1793.4014465825376)
timelength_network_burst_s                                                  0.408662
bursting_channels                                   [H2, K2, K3, O1, O2, P1, P2, R2]
number_of_bursting_channels                                                        8
number_burst_starts                                                                4
number_burst_ends                                                                  4
active_channels_total              [A11, B5, C10, C11, C14, C7, D

Funktion 1:

spikedic wird eingepasst, es wird ein spikedic mit zwischen start und stop gesliceten spikes zurückgegeben

In [ ]:
def cut_spikedic

In [14]:
active_channels = 0
spikedic_seconds = {}
for key in spikedic_MAD:
    sec_array = spikedic_MAD[key]*tick*scale_factor_for_second
    spikedic_seconds[key]=sec_array
    active_channels += 1
spikearray_seconds = np.asarray(list(spikedic_seconds.values()))  

TypeError: can't multiply sequence by non-int of type 'float'

In [27]:
def plot_networkburst_traces(network_bursts_dictionary, filebase):
    
    
    for burst in network_bursts_dictionary:
        plt.ioff()
        k = burst

        nb_start = float(k.split("(")[1].split(")")[0].split(',')[0])
        nb_stop = float(k.split("(")[1].split(")")[0].split(',')[1])

        limit0, limit1 = np.around(nb_start-0.5, decimals=2), np.around(nb_stop+0.5, decimals=2)


        # networkburst of interest
        nboi= network_bursts_dictionary[k]
        bursting_channels = nboi['bursting_channels']
        active_channels = nboi['active_channels']
        down_channels = list(network_bursts_dictionary[k]['channels_lfp_up'])
        up_channels = list(network_bursts_dictionary[k]['channels_lfp_up'])
        lfp_channels = up_channels + down_channels
        
   
        channels_of_interest = list(active_channels)

        channels_of_interest.sort()

        number_traces = len(channels_of_interest)

        fig = plt.figure(figsize=(10, number_traces))

        gs1 = gridspec.GridSpec(number_traces, 1)
        gs1.update(wspace=0.025, hspace=0.05) 
        axs = []
        
        
        # change to the subfolder containing the relevant bandpass and lowpass dic
        # relevant_subfolder = 
        # os.chdir()

        for i in range(1, number_traces+1):


            key = channels_of_interest[i-1]

            #no get all signals to plot and the lfp_down and lfp_ups
            bandpass_signal = bandpass_dic[key]
            # in the lowpass_dic there are still additional returns from the old butter filter function
            lowpass_signal = lowpass_dic[key]
            ups = lfp_ups[key]
            downs = lfp_downs[key]



            axs.append(fig.add_subplot(gs1[i-1]))


            axs[-1] = plt.plot(time_in_sec, bandpass_signal, c="#048ABF", linewidth = 0.5)
            axs[-1] = plt.plot(time_in_sec, lowpass_signal, c='#F20505', linewidth=1)
            #ax.spines['top'].set_visible(False)
            #ax.spines['right'].set_visible(False)
            #ax.spines['bottom'].set_visible(False)
            #ax.spines['left'].set_visible(False)
            #ax.get_xaxis().set_visible(False)
            frameon = False

            ax = plt.axvspan(nb_start, nb_stop, color='#C1D96C', alpha=0.1)

            for i in downs:
                ax = plt.axvspan(i[0], i[1], color='#5D7CA6', alpha=0.2)
            for i in ups:
                ax = plt.axvspan(i[0], i[1], color='#BF214B', alpha=0.2)



            # plt xlim for zooming in the time
            plt.xlim(limit0, limit1)
            plt.ylim(-70, 70)
            plt.yticks(fontsize='xx-small')
            plt.ylabel(key)

        fig.suptitle(filebase + ' network burst from '+str(limit0)+' to '+str(limit1))

        fig.savefig(
        filebase + '_lfp_and_bandpasssignal_cutout_from_' + str(limit0) +'_to_'+str(limit1)+'.png',
        bbox_inches='tight', dpi=300)
        plt.close()


In [37]:

# create folderlist of the temp directory
os.chdir(temp_dir)
folderlist = glob.glob('*'+filename+'*')


lfp_downs_dic = {}
lfp_downs_amp_dic = {}
lfp_ups_dic = {}
lfp_ups_amp_dic = {} 
#full_bandpass_dic = {}
#full_lowpass_dic = {}



In [38]:
folderlist

['190415_01_Cortex-synChR2-A_aCSF_nodrug_from_0_to_120_analyzed_on_02112021',
 '190415_01_Cortex-synChR2-A_aCSF_nodrug_from_1080_to_1200_analyzed_on_02112021',
 '190415_01_Cortex-synChR2-A_aCSF_nodrug_from_1200_to_1320_analyzed_on_02112021',
 '190415_01_Cortex-synChR2-A_aCSF_nodrug_from_120_to_240_analyzed_on_02112021',
 '190415_01_Cortex-synChR2-A_aCSF_nodrug_from_1320_to_1440_analyzed_on_02112021',
 '190415_01_Cortex-synChR2-A_aCSF_nodrug_from_1440_to_1560_analyzed_on_02112021',
 '190415_01_Cortex-synChR2-A_aCSF_nodrug_from_1560_to_1680_analyzed_on_02112021',
 '190415_01_Cortex-synChR2-A_aCSF_nodrug_from_1680_to_1800_analyzed_on_02112021',
 '190415_01_Cortex-synChR2-A_aCSF_nodrug_from_240_to_360_analyzed_on_02112021',
 '190415_01_Cortex-synChR2-A_aCSF_nodrug_from_360_to_480_analyzed_on_02112021',
 '190415_01_Cortex-synChR2-A_aCSF_nodrug_from_480_to_600_analyzed_on_02112021',
 '190415_01_Cortex-synChR2-A_aCSF_nodrug_from_600_to_720_analyzed_on_02112021',
 '190415_01_Cortex-synChR2-A_a

In [39]:
for i in folderlist:
    
    subdirectory = os.path.join(temp_dir, i)
    os.chdir(subdirectory)
    
    # join the timekey (we need a dictionary structure to join all the subparts
    # for long recordings)
    timekey = '_'.join(i.split('_')[-6:-3])

    # get the respective filenames
    lowpassfile = glob.glob('*lowpass*.npy')[0]
    bandpassfile = glob.glob('*bandpass*.npy')[0]
    
    lfp_down_file = glob.glob('*LFP_DOWNS*.npy')[1]
    lfp_up_file = glob.glob('*LFP_UPS*.npy')[1]
    
    lfp_amp_down_file = glob.glob('*LFP_Amplitudes_DOWNS*.npy')[0]
    lfp_amp_up_file = glob.glob('*LFP_Amplitudes_UPS*.npy')[0]
    
    cs_lfp_down_file = glob.glob('*cs_lfp_down*.npy')[0]
    cs_lfp_up_file = glob.glob('*cs_lfp_up*.npy')[0]
    
    cs_lfp_amp_down_file = glob.glob('*cs_lfp_amp*down*.npy')[0]
    cs_lfp_amp_up_file = glob.glob('*cs_lfp_amp*up*.npy')[0]



    # bandpass and lowpass signal
    lowpass_dic = np.load(lowpassfile, allow_pickle=True).item()
    bandpass_dic = np.load(bandpassfile, allow_pickle=True).item()
    
    # lfp deviations
    lfp_downs = np.load(lfp_down_file, allow_pickle=True).item()
    lfp_ups = np.load(lfp_up_file, allow_pickle=True).item()
    
    # lfp amplitudes
    lfp_amplit_downs = np.load(lfp_amp_down_file, allow_pickle=True).item()
    lfp_amplit_ups = np.load(lfp_amp_up_file, allow_pickle=True).item()
    
    # cs lfp deviations
    cs_lfp_downs = np.load(cs_lfp_down_file, allow_pickle=True).item()
    cs_lfp_ups = np.load(cs_lfp_up_file, allow_pickle=True).item()
    
    # cs lfp amplitudes
    cs_lfp_amplit_downs = np.load(cs_lfp_amp_down_file, allow_pickle=True).item()
    cs_lfp_amplit_ups = np.load(cs_lfp_amp_up_file, allow_pickle=True).item()
    
    lfp_downs_dic[timekey] = lfp_downs
    lfp_downs_amp_dic[timekey] = lfp_amplit_downs
    lfp_ups_dic[timekey] = lfp_ups
    lfp_ups_amp_dic[timekey] = lfp_amplit_ups 
   # full_lowpass_dic[timekey] = lowpass_dic
   # full_bandpass_dic[timekey] = bandpass_dic

In [43]:
def find_corresponding_signal_for_networkbursts(timestamps_networkburst, name_of_recording, folder_to_search):

    '''
    parameters:
    timestamps_networkburst = tuple, (a, b), a=start time of the burst in seconds, b = stop time of the burst in seconds
    dictionary_to_search = dictionary containtining any signal with k = timekey, v = signal
    
    '''
    start_networkburst = timestamps_networkbursts[0]
    stop_networkburst = timestamps_networkbursts[1]
    
    

SyntaxError: invalid syntax (Temp/ipykernel_7024/4009940153.py, line 1)

In [44]:
tsnwb = df.iloc[3]['network_burst_seconds']

In [45]:
tsnwb

'(27.22547789022808, 27.285476937341837)'

In [50]:
start_net = tsnwb[0]
stop_net = tsnwb[1]

os.chdir(temp_dir)

In [51]:
folderlist = glob.glob('*'+filename+'*')

In [52]:
folderlist

['190415_01_Cortex-synChR2-A_aCSF_nodrug_from_0_to_120_analyzed_on_02112021',
 '190415_01_Cortex-synChR2-A_aCSF_nodrug_from_1080_to_1200_analyzed_on_02112021',
 '190415_01_Cortex-synChR2-A_aCSF_nodrug_from_1200_to_1320_analyzed_on_02112021',
 '190415_01_Cortex-synChR2-A_aCSF_nodrug_from_120_to_240_analyzed_on_02112021',
 '190415_01_Cortex-synChR2-A_aCSF_nodrug_from_1320_to_1440_analyzed_on_02112021',
 '190415_01_Cortex-synChR2-A_aCSF_nodrug_from_1440_to_1560_analyzed_on_02112021',
 '190415_01_Cortex-synChR2-A_aCSF_nodrug_from_1560_to_1680_analyzed_on_02112021',
 '190415_01_Cortex-synChR2-A_aCSF_nodrug_from_1680_to_1800_analyzed_on_02112021',
 '190415_01_Cortex-synChR2-A_aCSF_nodrug_from_240_to_360_analyzed_on_02112021',
 '190415_01_Cortex-synChR2-A_aCSF_nodrug_from_360_to_480_analyzed_on_02112021',
 '190415_01_Cortex-synChR2-A_aCSF_nodrug_from_480_to_600_analyzed_on_02112021',
 '190415_01_Cortex-synChR2-A_aCSF_nodrug_from_600_to_720_analyzed_on_02112021',
 '190415_01_Cortex-synChR2-A_a